<a href="https://colab.research.google.com/github/vhppacheco/Analise-Ibovespa/blob/main/Ibovespa_Analysis_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega 1 - Análise Preliminar Ibovespa

#### pip install

In [ ]:
!pip install -q plotly

In [ ]:
!pip install -q yfinance

##### Instalação da Biblioteca Kora, que possibilita utilizar o Selenium de forma simplificada no Google Colab. [Link para acesso ao repositório](https://github.com/korakot/kora/blob/master/kora/selenium.py)

In [ ]:
!pip install -q kora

##Importanto as Bibliotecas

In [ ]:
from kora.selenium import wd
from time import sleep

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import yfinance as yf
import plotly.graph_objs as go

import os

import requests
from bs4 import BeautifulSoup

from google.colab import files
import warnings
warnings.filterwarnings('ignore')

## Webcrawling Carteira Ibovespa -16/02/2021

In [ ]:
def carteira_teorica(indice):
  url = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{}?language=pt-br'.format(indice.upper())
  wd.get(url)
  sleep(5)

  wd.find_element_by_id('segment').send_keys("Setor de Atuação")
  sleep(5)

  wd.find_element_by_link_text("Download").click()
  sleep(5)

  arquivo = !ls -1t *.csv
    
  return pd.read_csv(arquivo[0],sep=';',encoding='ISO-8859-1',engine='python',thousands='.', decimal=',', header= 1, index_col= False)

In [ ]:
ibov = carteira_teorica('ibov').iloc[:-2]

In [ ]:
#Obtendo do arquivo recém baixado e incorporando sua data de referência em uma coluna

arquivo = !ls -1t *.csv
arquivo_middle = arquivo[0].split('_')[1]
arquivo_date = arquivo_middle.split('.')[0]

ibov['Data Carteira'] = arquivo_date
ibov['Data Carteira'] = pd.to_datetime(ibov['Data Carteira'], errors='coerce')
ibov

In [ ]:
ibov.info()

In [ ]:
ibov.isna().sum()

In [ ]:
ativos_setor = ibov['Setor'].groupby(ibov['Setor']).count().sort_values(ascending=False)
ativos_setor = pd.DataFrame(ativos_setor).rename(columns={"Setor":"Nº de Ativos"})
ativos_setor = pd.merge(ativos_setor,ibov, how='left',on=['Setor']).drop_duplicates(subset=['Setor'])[['Setor','Nº de Ativos','Part. (%)Acum.']]
ativos_setor = ativos_setor.set_index('Setor')
ativos_setor

In [ ]:
y= ativos_setor['Nº de Ativos']
x = ativos_setor.index

fig = go.Figure(data=[go.Bar(x = x,
             y=y,
            text=y,
            textposition='auto',
            marker_color = 'blue'
        )])
fig.update_layout(height=500, width=1800,title_text='Nº de Ativos por Setor')
fig.show()

In [ ]:
y= ativos_setor['Part. (%)Acum.'].sort_values(ascending= False)
x = ativos_setor.index

fig = go.Figure(data=[go.Bar(x = x,
             y=y,
            text=y,
            textposition='auto',
            marker_color = 'red'
        )])
fig.update_layout(height=500, width=1800,title_text='Participação % por Setor')
fig.show()

## Webscraping Dados Fundamentalistas das Ações  do Ibovespa

##### O site utilizado como fonte é o Fundamentus. [Link do site](http://www.fundamentus.com.br/resultado.php)

In [ ]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [ ]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [ ]:
df_fund = pd.read_html(r.text,  decimal=',', thousands='.')[0]
df_fund = df_fund.rename(columns={"Papel":"Código"})

In [ ]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df_fund[coluna] = df_fund[coluna].str.replace('.', '')
  df_fund[coluna] = df_fund[coluna].str.replace(',', '.')
  df_fund[coluna] = df_fund[coluna].str.rstrip('%').astype('float') / 100

In [ ]:
df_fund

## Cruzando a Carteira do Ibovespa com os Indicadores Fundamentalistas

In [ ]:
ibov_fund = pd.merge(ibov,df_fund, how='left',on=['Código']).sort_values(by='P/L', ascending = True).reset_index().iloc[:,1:]
ibov_fund

In [ ]:
ibov_fund.isna().sum()

In [ ]:
ibov_fund.to_excel('Ibov+Fundamentos_17_02.xlsx')

In [ ]:
#Download na máquina local

arquivo = !ls -1t *.xlsx
files.download(arquivo[0])

#### Ler arquivo .xlsx caso precise

In [ ]:
ibov_fund = pd.read_excel('/content/Ibov+Fundamentos.xlsx').iloc[:,1:]
ibov_fund

## Obtenção dados CVM - Companhias Abertas

In [ ]:
r_cvm = requests.get('http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv')

rows = [i.strip().split(';') for i in r_cvm.text.split('\n')]
df_cvm = pd.DataFrame(rows[1:], columns=rows[0])[:-1]
df_cvm

In [ ]:
df_cvm.tail(3)

In [ ]:
df_cvm['DT_REG'] = pd.to_datetime(df_cvm['DT_REG'], errors='coerce')

In [ ]:
df_cvm.info()

## Cruzamento de bases Ibovespa+Fundamentos e CVM

### Webscraping Infomoney (busca por CNPJ como chave única entre as bases do IBOV+Fundamentos e CVM)

In [ ]:
url2 = 'https://www.infomoney.com.br/minhas-financas/confira-o-cnpj-das-acoes-negociadas-em-bolsa-e-saiba-como-declarar-no-imposto-de-renda/'
response2 = requests.get(url2)

html2 = BeautifulSoup(response2.text, "html.parser")
html2.find_all('table')

In [ ]:
infomoney = pd.read_html(response2.text,header=0)[0]
infomoney

In [ ]:
infomoney.to_excel('cnpj_infomoney.xlsx')

In [ ]:
#Download na máquina local

arquivo = !ls -1t *.xlsx
files.download(arquivo[0])

In [ ]:
infomoney = pd.read_excel('cnpj_infomoney.xlsx').iloc[:,1:]
infomoney

In [ ]:
infomoney = infomoney.rename(columns={"Nome de pregão":"Ação"})

In [ ]:
ibov_fund_infomoney = pd.merge(ibov_fund, infomoney, how='left',on=['Ação']).reset_index().iloc[:,1:]
ibov_fund_infomoney

In [ ]:
ibov_fund_infomoney.isna().sum()

In [ ]:
filtro_nan= ibov_fund_infomoney[ibov_fund_infomoney['CNPJ'].isnull()]
filtro_nan

In [ ]:
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].str.replace('.', '')
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].str.replace('/', '')
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].str.replace('-', '')
ibov_fund_infomoney['CNPJ'] = ibov_fund_infomoney['CNPJ'].astype(int)
ibov_fund_infomoney

In [ ]:
df_cvm = df_cvm.rename(columns={"CNPJ_CIA":"CNPJ"})
df_cvm['CNPJ'] = df_cvm['CNPJ'].str.replace('.', '')
df_cvm['CNPJ'] = df_cvm['CNPJ'].str.replace('/', '')
df_cvm['CNPJ'] = df_cvm['CNPJ'].str.replace('-','')
df_cvm['CNPJ'] = df_cvm['CNPJ'].astype(int)
df_cvm

In [ ]:
ibov_fund_cvm =  pd.merge(ibov_fund_infomoney,df_cvm, how='left',on=['CNPJ']).sort_values(by='P/L', ascending = True).reset_index().iloc[:,1:]
ibov_fund_cvm

In [ ]:
filtro_nan2= ibov_fund_cvm[ibov_fund_cvm['CNPJ'].isnull()]
filtro_nan2

In [ ]:
ibov_fund_cvm.columns

In [ ]:
ibov_fund_cvm = ibov_fund_cvm[['Setor', 'Código', 'Ação', 'Tipo', 'Qtde. Teórica', 'Part. (%)',
       'Part. (%)Acum.', 'Data Carteira', 'DT_REG', 'Cotação', 'P/L', 'P/VP', 'PSR',
       'Div.Yield', 'P/Ativo', 'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq',
       'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC',
       'ROE', 'Liq.2meses', 'Patrim. Líq', 'Dív.Brut/ Patrim.',
       'Cresc. Rec.5a']]

ibov_fund_cvm

In [ ]:
ibov_fund_cvm['DT_REG'] = pd.to_datetime(ibov_fund_cvm['DT_REG'], errors='coerce')
ibov_fund_cvm.info()

In [ ]:
ibov_fund_cvm = ibov_fund_cvm.rename(columns={"DT_REG":"Data Referência IPO"})
ibov_fund_cvm.to_excel('Base_Ibov_final.xlsx')

In [ ]:
#Download na máquina local

arquivo = !ls -1t *.xlsx
files.download(arquivo[0])